# <center> Hyperparameter Tuning for Generative Adversarial Network (NLS-KDD) <center/>

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import preprocessing
from classifiers import *
from utils import *
from matplotlib import pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from scipy.stats import norm
from tensorflow.keras.losses import kullback_leibler_divergence

Using TensorFlow backend.


## Read Data

In [ ]:
train,test, label_mapping = preprocessing.get_data(encoding="Label")
x_train,y_train = train.drop("label",axis=1),train.label.values
x_test , y_test =  test.drop("label",axis=1),test.label.values

Scaler = StandardScaler()
x_train = Scaler.fit_transform(x_train)
x_test = Scaler.transform(x_test)

att_ind = np.where(train.label != label_mapping["normal"])[0]

x = x_train[att_ind]

## Set Hyperparams

In [ ]:
rand_dim = 32
base_n_count = 100

combined_ep = 2000
batch_size = 128 if len(x) > 128 else len(x)

ep_d = 1
ep_g = 2
learning_rate = 0.0001

## Generator & Discriminator Definitions

In [ ]:
def get_discrimnator(data_dim, min_num_neurones,n_layers, lr ,optimizer ,activation_f)
    model = tf.keras.models.Sequential(name='Discriminator')
    
    model.add(Dense(min_num_neurones*n_layers, activation=activation_f,input_dim = data_dim))
    for n in reversed(range(1,n_layers)):
        model.add(Dense(n*min_num_neurones, activation = activation_f))
        
    model.add(Dense(1,activation="sigmoid"))
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    K.set_value(model.optimizer.lr, lr)
    
    return model

def get_generator(data_dim, min_num_neurones,noise_dim,n_layers, lr ,optimizer ,activation_f)
    model = tf.keras.models.Sequential(name='Generator')
    
    model.add(Dense(min_num_neurones, activation=activation_f,input_dim = noise_dim ))
    for n in range(1,n_layers):
        model.add(Dense(n*min_num_neurones, activation = activation_f))
        
    model.add(Dense(data_dim,activation=activation_f))
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    K.set_value(model.optimizer.lr, lr)
    
    return model